In [ ]:
import pandas as pd 
import ipywidgets as widgets
import datetime
import sqlalchemy
import psycopg2
import golf_functions

This notebook is to add golf course tee infromation. Tee infromation for a course will need to be in the course_info table to add hole infromation for the course. 

To enter new course tee information you will need:
- par
- tee
- total_distance
- F9_distance
- B9_distance
- rating
- slope

In [ ]:
course_select_query = '''
Select cn.course_id
    ,cn.course_name
    ,cv.course_version_id
from "Courses"."course_name" as cn
left join (select course_id, MAX(course_version_id) as course_version_id from "Courses"."course_version" group by course_id) as cv on cn.course_id = cast(cv.course_id as int)
'''

In [ ]:
courses = golf_functions.pull_from_sql(course_select_query)

In [ ]:
course_select = widgets.Dropdown(options = list(courses['course_name']))
course_select

In [ ]:
course_info_entry = courses[courses['course_name'] == course_select.value]

In [ ]:
# labels and widgets for data entry
par_lbl = widgets.Label('Par')
par_ent = widgets.Text()
tee_lbl = widgets.Label('Tee Name')
tee_ent = widgets.Text()
total_distance_lbl = widgets.Label('Total Distance')
total_distance_ent = widgets.IntText()
F9_distance_lbl = widgets.Label('Front 9 Distance')
F9_disance_ent = widgets.IntText()
B9_distance_lbl = widgets.Label('Back 9 Distance')
B9_distance_ent = widgets.IntText()
rating_lbl = widgets.Label('Rating')
rating_ent = widgets.FloatText()
slope_lbl = widgets.Label('Slope')
slope_ent = widgets.IntText()

In [ ]:
# display() is used for multiple entries
display(par_lbl)
display(par_ent)
display(tee_lbl)
display(tee_ent)
display(total_distance_lbl)
display(total_distance_ent)
display(F9_distance_lbl)
display(F9_disance_ent)
display(B9_distance_lbl)
display(B9_distance_ent)
display(rating_lbl)
display(rating_ent)
display(slope_lbl)
display(slope_ent)

In [ ]:
# dict used to make df needed for load_sql
course_info_dict = {
    'par':par_ent.value,
    'tee':tee_ent.value,
    'total_distance':total_distance_ent.value,
    'f9_distance':F9_disance_ent.value,
    'b9_distance':B9_distance_ent.value,
    'rating':rating_ent.value,
    'slope':slope_ent.value,
    'course_name':course_select.value
}
df = pd.DataFrame([course_info_dict],columns=course_info_dict.keys())
course_info_entry_final = pd.merge(course_info_entry,df, on='course_name')
#double checking entry
print('Is the correct course information')
print(course_info_entry_final['course_name'][0])
course_info_entry_final.drop('course_name', axis=1, inplace = True)
course_info_entry_final

If the above course information is correct please run the next cell. 

In [ ]:
golf_functions.load_sql(course_info_entry_final, 'Courses', 'course_info')